In [ ]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import lazypredict
from lazypredict.Supervised import LazyClassifier

### The netherlands

In [ ]:
netherlands = pd.read_excel('../data/42256_2020_253_MOESM1_ESM.xlsx', engine = 'openpyxl')

df = netherlands[["Age", "Gender", "LD", "CRP", "Percentage lymphocytes", "Survival/death"]]
df = df.assign(death = np.where(df["Survival/death"] == 'Alive', 0, 1))
df = df[["Age", "Gender", "LD", "CRP", "Percentage lymphocytes", "death"]]
X = df.drop("death", axis = 1)
y_netherlands = df.death
netherlands = df
y_netherlands = df.death
netherlands = netherlands.iloc[:,2:-2]
netherlands['death'] = y_netherlands
netherlands['from'] = 'netherlands'
netherlands = netherlands.rename(columns = { 'Percentage lymphocytes':'Lymphocytes', 'LD':'LDH'})

### China

In [41]:
import sys
sys.path.append("..")
china = pd.read_excel('../data/time_series_375_prerpocess_en.xlsx', engine = 'openpyxl')

from raport_v1.utils_features_selection import data_read_and_split
X_data_all_features, Y_data, x_col = data_read_and_split()
cheng = {'乳酸脱氢酶':'LDH', '超敏C反应蛋白':'CRP', '淋巴细胞(%)':'Lymphocytes'}
blood_df = X_data_all_features[list(cheng.keys())]
blood_df.columns = list(cheng.values())
china = blood_df
china['death'] = Y_data
china['from'] = 'china'
china = china.reset_index(drop = True)

### NY

In [43]:
ny = pd.read_csv('~/Downloads/Yan_reply_First_last_wtime.csv')
y_ny = ny.iloc[:,1]
newyork = ny.iloc[:,[5,6,7]]
newyork = newyork.rename(columns = {'First_LDH':'LDH',
                                      'First_CRP':'CRP',
                                      'First_Lymph':'Lymphocytes'})
newyork['from'] = 'newyork'
newyork['death'] = y_ny

## Combining together

In [44]:
nations = china.append([newyork, netherlands])
nations['death'] = nations.death.astype('int')
nations = nations.reset_index(drop=True)
# nations.to_csv("all.csv")

In [45]:
# nations = pd.read_csv('all.csv')
X = nations[['LDH','CRP','Lymphocytes']]
X_train, X_test, y_train, y_test = train_test_split(X, nations.death, test_size = 0.25, random_state = 42)

clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models.to_csv("LazyClassifier_results.csv")

print(models)